# **Programa de Pós-Graduação em Computação - INF/UFRGS**

---


### Disciplina CMP263 - Aprendizagem de Máquina
#### *Profa. Mariana Recamonde-Mendoza (mrmendoza@inf.ufrgs.br)*
<br>

---
***Observação:*** *Este notebook é disponibilizado aos alunos como complemento às aulas  e aos slides preparados pela professora. Desta forma, os principais conceitos são apresentados no material teórico fornecido. O objetivo deste notebook é reforçar os conceitos e demonstrar questões práticas no uso de algoritmos e estratégias de avaliação em Aprendizado de Máquina.*


---

##**Tópico: Naïve Bayes**


**Objetivos do notebook:**
-  Demonstrar o processo de treinamento e avaliação de modelos com o algoritmo Naïve Bayes e a técnica Holdout;
- Explorar o modelo Naïve Bayes, em termos de parâmetros estimados e fronteira de decisão gerada.

In [ ]:
## Naive Bayes com dados de câncer de mama

# 1. Importando bibliotecas
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from matplotlib.colors import ListedColormap

# 2. Carregando o dataset a partir do sklearn
cancer = load_breast_cancer()
X = cancer.data
y = cancer.target

print("Features:", cancer.feature_names[:10], "...")
print("Classes:", cancer.target_names)

# 3. Dividir em treino e teste, de forma estratificada
# O uso do random_state visa garantir reprodutibilidade dos experimentos.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify = y)


In [ ]:
# 4. Treinar modelo Naive Bayes Gaussiano, com configuração padrão de hiperparâmetros
# Na etapa de treinamento, o algoritmo estima todas as probabilidades envolvidas na tomada de decisão
nb = GaussianNB()
nb.fit(X_train, y_train)


In [ ]:
import pandas as pd

# Número de instâncias por classe
print("Número de instâncias por classe:", nb.class_count_)

# Probabilidade a priori das classes
print("Probabilidades a priori:", nb.class_prior_)

# Médias das features por classe
df_means = pd.DataFrame(nb.theta_, columns=cancer.feature_names, index=cancer.target_names)
print("\nMédias estimadas por classe:")
display(df_means)

# Variâncias das features por classe
df_vars = pd.DataFrame(nb.var_, columns=cancer.feature_names, index=cancer.target_names)
print("\nVariâncias estimadas por classe:")
display(df_vars)


In [ ]:
# 5. Avaliar modelo
y_pred = nb.predict(X_test)
print("Acurácia:", accuracy_score(y_test, y_pred))
print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred, target_names=cancer.target_names))

cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', xticklabels=cancer.target_names, yticklabels=cancer.target_names, cmap='Blues')
plt.xlabel("Predito")
plt.ylabel("Real")
plt.title("Matriz de Confusão")
plt.show()


In [ ]:
#Como referência, vamos avaliar o desempenho nos dados de treino, para averiguar o risco de overfitting
y_pred_train = nb.predict(X_train)
print("Acurácia:", accuracy_score(y_train, y_pred_train))

In [ ]:
# 6. Mostrar probabilidades em alguns exemplos do teste
print("Exemplos de probabilidades preditas:")
for i in range(5):
    probs = nb.predict_proba([X_test[i]])[0]
    print(f"Exemplo {i}: Probabilidades = {probs}, Classe real = {cancer.target_names[y_test[i]]}, Classe predita = {cancer.target_names[y_pred[i]]}")


In [ ]:
# 7. Plotar fronteira de decisão (usando só duas features para visualização)
# Vamos escolher as duas primeiras features para simplificação
X_vis = X_train[:, :2]
X_test_vis = X_test[:, :2]

nb_vis = GaussianNB()
nb_vis.fit(X_vis, y_train)

# Criar grid para plotar
x_min, x_max = X_vis[:, 0].min() - 1, X_vis[:, 0].max() + 1
y_min, y_max = X_vis[:, 1].min() - 1, X_vis[:, 1].max() + 1
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200), np.linspace(y_min, y_max, 200))

Z = nb_vis.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

plt.figure(figsize=(8,6))
plt.contourf(xx, yy, Z, alpha=0.3, cmap=ListedColormap(["#FFAAAA", "#AAFFAA"]))
plt.scatter(X_vis[:, 0], X_vis[:, 1], c=y_train, edgecolor='k', cmap=ListedColormap(["#FF0000", "#00FF00"]), s=30)
plt.xlabel(cancer.feature_names[0])
plt.ylabel(cancer.feature_names[1])
plt.title("Fronteira de decisão do Naive Bayes (duas features)")
plt.show()


In [ ]:
from sklearn.decomposition import PCA

# Reduzindo para 2 componentes
pca = PCA(n_components=2)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Treinar NB no espaço reduzido
nb_pca = GaussianNB()
nb_pca.fit(X_train_pca, y_train)

# Grid para plotar
x_min, x_max = X_train_pca[:, 0].min() - 1, X_train_pca[:, 0].max() + 1
y_min, y_max = X_train_pca[:, 1].min() - 1, X_train_pca[:, 1].max() + 1
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200), np.linspace(y_min, y_max, 200))

Z = nb_pca.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

plt.figure(figsize=(8,6))
plt.contourf(xx, yy, Z, alpha=0.3, cmap=ListedColormap(["#FFAAAA", "#AAFFAA"]))
plt.scatter(X_train_pca[:, 0], X_train_pca[:, 1], c=y_train, edgecolor='k', cmap=ListedColormap(["#FF0000", "#00FF00"]), s=30)
plt.xlabel("PCA 1")
plt.ylabel("PCA 2")
plt.title("Fronteira de decisão (NB em espaço PCA das 30 features)")
plt.show()
